## Name: Lam Kam Yuen
## Date: 8/13/2024
## Task: ASSET MANAGEMENT PRACTICE PROJECT 01 


## Understanding of topics and tools

For the first task of the project, I understand the meaning of MVP model, the method of using python to calculate return and risk. Also I also learn how to calculate the minimum variance and maximum Sharpe.
However, for the second task, I used a little time to understand the rolling backtesting, especially on controlling the roll back period and out of sample testing. Actually I need to read the notes when doing the project.


## Strengths and weakness:	
Strengths:
	I understanding how to use pandas and numpy library. 
 Weakness:
	For me, use object-oriented methods (Class) to write programs is my weakness.


## Experiences:
	The first task I have confidence to finish it. The second task, I need to rewrite it one time as the rolling back method I use doesn't seem to get a correct answer. 


## import necessary library


In [1]:
import math
import numpy as np
import pandas as pd
from pylab import plt
from scipy.optimize import minimize
plt.style.use('seaborn-v0_8')
np.set_printoptions(suppress=True, precision=4)
pd.options.display.float_format = '{:.5f}'.format
#Create a FinancialData class:


## Create a FinancialData class:
    initial a class
    1) download data from csv url
    2) prepare the data for calcualte model
    3) use log return 
    

In [ ]:
class FinancialData:
    url = 'https://hilpisch.com/indices_eikon_eod_data.csv'
    def __init__(self):
        self.financialdata =None
        self.returns = None
        self.downloadData()
        self.prepare_data()
        self.no_assets = len(self.financialdata.columns)
    def downloadData(self):
        self.financialdata = pd.read_csv(self.url, index_col =0, parse_dates=True)
    def prepare_data(self):
        self.returns = np.log(self.financialdata /self.financialdata.shift(1)).dropna()
        
    

## MVP Class
    init()
    portfolio_return()
    portfolio_variance()
    portfolio_volatility()
    portfolio_sharpe()
    max_sharpe_portfolio()
    min_volatility_portfolio()
    _set_bounds_constraints()
    _return_result()

In [ ]:
class MVPmodel(FinancialData):
    def __init__(self):
        super().__init__()
        self.weights = self.no_assets * [1/ self.no_assets]
    def portfolio_return(self,weights,days=252):
        return np.dot(self.returns.mean(),weights)*days
    def portfolio_variance(self, weights,days =252):
        return np.dot(weights, np.dot(self.returns.cov(), weights)) * days
    def portfolio_volatility(self,weights,days=252):
        return math.sqrt(self.portfolio_variance(weights,days))
    def portfolio_sharpe(self,weights, days=252):
        shape = (self.portfolio_return(weights,days) /
                 self.portfolio_volatility(weights,days))
        return shape
    def _set_bounds_constraints(self, bnds, cons):
        if bnds is None:
            self.bnds = self.no_assets*[(0,1)]
        else:
            self.bnds = bnds
        if cons is None:
            self.cons = {'type': 'eq', 'fun': lambda weights: weights.sum() - 1}
        else:
            self.cons = cons
    #optimization methods to
    #derive the minimum variance portfolio as well as the maximum Sharpe
    #portfolio.
             
    def max_sharpe_portfolio(self,bnds=None, cons=None):
        self._set_bounds_constraints(bnds,cons)
        sharpe_f = lambda weights: -self.portfolio_sharpe(weights)
        opt = minimize(sharpe_f, self.weights,bounds=self.bnds,
                       constraints = self.cons)
        self.results =self._return_result(opt, 'Maximum Shape')
        return self.results
    def min_volatility_portfolio(self,bnds=None, cons=None):
        self._set_bounds_constraints(bnds,cons)
        opt = minimize(self.portfolio_volatility,self.weights,bounds=self.bnds,
                       constraints = self.cons)
        self.results =self._return_result(opt, 'Minimum Volatility')
        return self.results
    def _return_result(self,opt,kind):
        ret = self.portfolio_return(opt['x'])
        vol = self.portfolio_volatility(opt['x'])
        sharpe = self.portfolio_sharpe(opt['x'])
        weights= pd.DataFrame(opt['x'], index =self.financialdata.columns,columns=['weights',])
        res ={'kind': kind, 'weights': weights.round(7), 'return' : ret, 
              'volatiltiy': vol, 'sharpe' : sharpe}
        return res
    #optimization methods to
    #derive the minimum variance portfolio as well as the maximum Sharpe
    #portfolio.
    

In [ ]:
mvpmodel = MVPmodel()

In [ ]:
#mvpmodel.financialdata.columns

In [ ]:
mvpmodel.portfolio_return(mvpmodel.weights)

In [ ]:
mvpmodel.portfolio_volatility(mvpmodel.weights)

In [ ]:
mvpmodel.portfolio_sharpe(mvpmodel.weights)

In [ ]:
mvpmodel.min_volatility_portfolio()

In [ ]:
mvpmodel.max_sharpe_portfolio()

## Class RollbackTesting
    1) inherit MVPmodel
    2) control lookback period
    3) roll the date from start_date to end date
    

In [ ]:
class RollbackTesting(MVPmodel):
    def __init__(self,lookback_period):
        super().__init__()
        self.lookback_period = lookback_period
    def backtest(self, type_of_test,start_date, end_date, rebalance_period ='1W'):
        dates = pd.date_range(start=start_date, end=end_date, freq= rebalance_period)
        portfolio_returns =[]
        cal_looback = int(self.lookback_period / 5)
        org_returns = self.returns
        for i in range(cal_looback ,len(dates) -1):        
            train_end = dates[i]
            test_start = dates[i + 1]
            #print(train_end)
            test_end = dates[i + 2] if i + 2 < len(dates) else end_date
            train_data = org_returns[:train_end][-self.lookback_period:]
            test_data = org_returns[test_start:test_end]
            #rint(f'{test_start}:{test_end}:{len(test_data)}')  
            self.returns = train_data
            if  type_of_test == 'Maximum Sharpe':
                weights = self.max_sharpe_portfolio()['weights'].values.flatten()
            elif type_of_test == 'Minimum Volatility':
                weights = self.min_volatility_portfolio()['weights'].values.flatten()
            if len(test_data) > 0 :
                test_return = np.sum(weights * test_data.mean())
                portfolio_returns.append(test_return)
            #else:
                #print(test_start)
        performance_df = pd.DataFrame(portfolio_returns,  columns=['Portfolio Return'])
        performance_df['Cumulative Return'] = (1 + performance_df['Portfolio Return']).cumprod() - 1 
        self.returns = org_returns 
        return performance_df
    
    def annualized_return(self, performance_df):
        """
        Calculate the annualized return of the portfolio.
        :param performance_df: DataFrame of portfolio performance
        :return: Annualized return
        """
        total_return = performance_df['Cumulative Return'].iloc[-1]
        num_years = len(performance_df) / 52  # Assuming weekly returns
        return (1 + total_return) ** (1 / num_years) - 1
    
    
    def plot_backtest(self, backtest_results):
        """
        Plot the results of the backtest.
        
        Parameters:
        backtest_results (pandas.DataFrame): Results of the backtest.
        """
        plt.figure(figsize=(12, 6))
        backtest_results['Cumulative Return'].plot()
        plt.title('Backtest Portfolio Value')
        plt.xlabel('Date')
        plt.ylabel('Portfolio Value')
        plt.grid()
        plt.show()

In [ ]:
rollbacktest = RollbackTesting(504)

In [ ]:
rollbacktest.returns.info()

In [ ]:

type_of_test = 'Maximum Sharpe'
performance_df = rollbacktest.backtest(type_of_test, start_date='2016-01-01', end_date='2020-01-01')

# Calculate the annualized return
annualized_return = rollbacktest.annualized_return(performance_df)

print(f'Annualized Return of {type_of_test}:{annualized_return*100:.2f}%', )
#print(performance_df)

type_of_test = 'Minimum Volatility'
performance_df = rollbacktest.backtest(type_of_test, start_date='2016-01-01', end_date='2020-01-01')

# Calculate the annualized return
annualized_return = rollbacktest.annualized_return(performance_df)

print(f'Annualized Return of {type_of_test}:{annualized_return*100:.2f}%', )
#print(performance_df)


In [ ]:
rollbacktest.plot_backtest(performance_df)